# Collect and show metrics in jupyter dashboard

In this example we will learn how to collect metrics using Toloka-kit and
display it right inside this jupyter notebook.

In [ ]:
%%capture
!pip install toloka-kit==0.1.26
!pip install crowd-kit==1.0.0

import getpass

import toloka.metrics as metrics
import toloka.client as toloka
from toloka.metrics.jupyter_dashboard import Chart, DashBoard


In [ ]:
toloka_client = toloka.TolokaClient(getpass.getpass('Enter your OAuth token: '), 'PRODUCTION') # Or switch to 'SANDBOX'
print(toloka_client.get_requester())

For this example we will run pipeline from [Streaming pipeline example](https://github.com/Toloka/toloka-kit/tree/main/examples/6.streaming_pipelines/streaming_pipelines.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/6.streaming_pipelines/streaming_pipelines.ipynb).
If you are running this jupyter notebook in colab please download necessary script with the following line of code:

In [ ]:
!wget --quiet --show-progress "https://raw.githubusercontent.com/Toloka/toloka-kit/main/examples/metrics/find_items_pipeline.py"

In [ ]:
from find_items_pipeline import FindItemsPipeline
pipeline = FindItemsPipeline(client=toloka_client)

Create projects and pools needed for pipeline.

In [ ]:
pipeline.init_pipeline()

## Configuring jupyter dashboard

Let's create dashboard for all pools in the pipeline. For this example we will collect
a number of submitted assignments, accepted assignments and total expenses for each pool. All available metrics can be found in the [documentation](https://toloka.ai/en/docs/toloka-kit/reference/toloka.metrics.metrics.BaseMetric).

In [ ]:
new_dash = DashBoard(
    [
        # Manually configured charts may contain several metrics and draw all their lines in one chart.
        # You must clearly specify chart name
        # Be careful, if you add same metric type with default line names, you get pair of lines with same names.
        Chart(
            'Assignments in pools',
            # Assignments in pools. We will track submitted assignments and accepted assignments counts for every pool.
            [metrics.AssignmentsInPool(
                pipeline.verification_pool.id,
                submitted_name='verification_pool.submitted_assignments',
                accepted_name='verification_pool.accepted_assignments',
            ),
            metrics.AssignmentsInPool(
                pipeline.find_items_pool.id,
                submitted_name='find_items_pool.submitted_assignments',
                accepted_name='find_items_pool.accepted_assignments',
            ),
            metrics.AssignmentsInPool(
                pipeline.sbs_pool.id,
                submitted_name='sbs_pool.submitted_assignments',
                accepted_name='sbs_pool.accepted_assignments',
            )]
        ),
        Chart(
            'Pools expenses',
            [# Budget spent for every pool
            metrics.SpentBudgetOnPool(
                pipeline.verification_pool.id,
                'verification_pool.expenses'
            ),
            metrics.SpentBudgetOnPool(
                pipeline.find_items_pool.id,
                'find_items_pool.expenses'
            ),
            metrics.SpentBudgetOnPool(
                pipeline.sbs_pool.id,
                'sbs_pool.expenses'
            ),]
        ),
    ],
    update_seconds=2,  # just example. In real dashboards it's better to drop this parameter
    header='Find items pipeline dashboard',
)

metrics.bind_client(new_dash.metrics, toloka_client)

Start dashboard. You can run other cells, while this dashboard is tracking metrics.

In [ ]:
new_dash.run_dash()


---
**NOTE**
- This DashBoard is useful only for fast and pretty online demonstration in jupyter notebooks. For real monitoring system, please use `toloka.metric.MetricCollector` (see [Graphite example](https://github.com/Toloka/toloka-kit/tree/main/examples/metrics/graphite.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/tree/main/examples/metrics/graphite.ipynb))
- DashBoard is not saved with jupyter notebook. It uses `IPython.lib.display.IFrame`. So if you save notebook and share .ipynb file, there will be no dashboard images.
- DashBoard works with current instance, so you can not share it.
---


## Running pipeline

Let's try to launch our pipeline and see metrics updated. Metrics will be simultaneously drawn in the above dashboard.

⚠️ **Be careful**:
real projects will be created and money will be spent in case of running in production environment! ⚠️

In [ ]:
import asyncio
# Google Colab is using a global event pool,
# so in order to run our pipeline we have to apply nest_asyncio to create an inner pool
if 'google.colab' in str(get_ipython()):
    import nest_asyncio, asyncio
    nest_asyncio.apply()
    asyncio.get_event_loop().run_until_complete(pipeline.run())
else:
    await pipeline.run()

If you want to stop dashboard call this:

In [ ]:
new_dash.stop_dash()

After pipeline completion your dashboard should contain charts similar to this:

<table  align="center">
  <tr><td>
    <img src="./img/assignments_in_pool_dash.png" width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> Assignments in pools metric chart.
  </td></tr>
</table>

<table  align="center">
  <tr><td>
    <img src="./img/pool_expenses_dash.png" width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 2.</b> Pools expenses metric chart.
  </td></tr>
</table>

---

## Tips for using Toloka-kit metrics

- If you need several dashboard, create several instances of DashBoard and run it in several ports like:
```Python
    dash_for_requester1 = DashBoard(toloka_client1, [...])
    dash_for_requester2 = DashBoard(toloka_client2, [...])
```
And then run these instances in different cells on different ports:</br></br>
First cell:
```Python
    dash_for_requester1.run_dash(port='8081')
```
Second cell:
```Python
    dash_for_requester2.run_dash(port='8082')
```
- You can use same dashboard for metrics from different clients:
```Python
    toloka_client_1 = toloka.TolokaClient(<FIRST CLIENT TOKEN>, 'PRODUCTION')
    toloka_client_2 = toloka.TolokaClient(<SECOND CLIENT TOKEN>, 'PRODUCTION')

    my_dash = DashBoard(
        [
            metrics.Balance(toloka_client=toloka_client_1),
            metrics.Balance(toloka_client=toloka_client_2),
            Chart(
                'Balance for both clients',
                [
                    metrics.Balance(balance_name='first client', toloka_client=toloka_client_1),
                    metrics.Balance(balance_name='second client', toloka_client=toloka_client_2),
                ]
            )
        ],
        header='Dashboard for several clients',
    )
```
You **don't** need to call `bind_client` afterwards because you have already binded clients in each metric.
